# FINAL PROJECT
# By: Hagar Zaguri
***

# Introduction

The goal of the project is to increase the profit of an online store that sells home products by studing the product categories as well as the price range.

Pricing products and services that the business offers is the main mechanism for generating profits. When determining a price for a product many variables have to be taken into account such as gross cost of the product, operating expenses of the business, branding, comparison to competitors and more. There are several main approaches when setting a price of a product, but the main formula is : the higher the product price the higher the gross profit from a single sale will increase but fewer people will buy. As the price goes down the gross profit goes down but the number of sales of each product increase and so the total profit increase. The goal of the business is to find the optimal price so that it will earn the most. There is also a strategy called Loss leader pricing which involves selecting one or more retail products to be sold below cost – at a loss to the retailer – in order to get customers in the door.

Since the dataset does not contain an information of the gross cost of each prudoct, I will set the 'base price' for each product as the maximum price at which the product was sold and I will refer to the price range of the product accordingly.

In this project I will study the different categoiries and the price range of the product. At the end I will draw my conclusions an write recommendations to the buisness.

<b>The original datasets contains the following columns:</b>

* InvoiceNo` — order identifier
* StockCode` — item identifier
* Description` — item name
* Quantity`
* InvoiceDate` — order date
* UnitPrice` — price per item
* CustomerID`

[Link to presentation](https://drive.google.com/file/d/1v2wn2t7o_NJbTiAQhy82DT5yhasvRnzS/view?usp=sharing)<br>
[Tableau dashboard](https://public.tableau.com/views/Finalproject_16509691268200/Dashboard1?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link)

<b>Pricing articles:</b>
* [Optimal pricing models: 3 strategies for optimization](https://quickbooks.intuit.com/r/midsize-business/pricing-strategy-models/) by Susan Guillory
* [How to price a product and introduce discounts](https://www.productmarketingalliance.com/how-to-price-a-product-and-introduce-discounts/) by Lawrence Chapman
* [The Risks, Benefits, and Point of a Loss Leader Pricing Strategy](https://blog.hubspot.com/sales/loss-leader-pricing)
* [How to Get the Price Right,Tools for optimal price setting](https://medium.com/teconomics-blog/how-to-get-the-price-right-9fda84a33fe5) by Emily Glassberg Sands



## Preproccessing data:

### Data overview

In [ ]:
import pandas as pd
import datetime as dt
from scipy import stats as st
import math as mth
import matplotlib.pyplot as plt 
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns
import plotly.figure_factory as ff
from plotly.figure_factory import create_distplot

from sklearn.cluster import KMeans
from nltk.stem import PorterStemmer
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings("ignore")

In [ ]:
try:
    data = pd.read_csv('ecommerce_dataset_us.csv', sep = '\t')
except:
    data = pd.read_csv('/datasets/ecommerce_dataset_us.csv', sep = '\t')

In [ ]:
# make a copy of the original dataset
data_1 = data

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.describe(include='all')

<b> There are missing values in the 'Description' and Costumer_ID' columns. Also, there are negative values in the 'Quantity' and 'UnitPrice' columns, which should contain innegative values. I dealed with all these columns in the next stage.</b>

### Dealing with missing and odd values

<b> I started with checking the missing values in the Description column</b>

In [ ]:
data.query('Description.isnull()').head(10)

<b> There is a certain probability that the reason for the missing values in column A also causes the missing values in column B. 
<br> 
    
I filled in the missing values in the 'Description' column by using descriptions of products with the same stock code.</b>



In [ ]:
# Creating a  table that contains only rows with non-missing values in the Description column
no_missing_desc  =data.query('Description.notnull()')
# Create a table that contains description for every stockcode that is in the table above
top_desc = no_missing_desc .groupby('StockCode').agg({'Description':'first'}).reset_index()
top_desc.columns= ['StockCode', 'first_desc']
top_desc.head()

In [ ]:
# adding the first descrption column to main table, only for filling missing values
data = data.merge(top_desc, how='left', on='StockCode')
data['Description'] = data.Description.fillna(data['first_desc'])

In [ ]:
print('Number of unique products:', data.StockCode.nunique())
print('Number of  products with missing description (after filling in):', data.query('Description.isnull()').StockCode.nunique())
print('Share of products with no description:',  data.query('Description.isnull()').StockCode.nunique()/ data.StockCode.nunique())
print()
print('size od data:', len(data))
print('Number of missing values in the description column:', len(data.query('Description.isnull()')))
print('Share of missing values in the description column:', len(data.query('Description.isnull()'))/len(data))

<b> The percentage of rows with the missing values in the description column is very small. Since in this project the description of the product is a very important, I decided to drop these rows.</b>

In [ ]:
# dropping missing values of the description column
data = data.dropna(subset=['Description'])
# dropping 'first_desc' column (added only for filling missing values)
data = data.drop(labels=['first_desc'], axis=1)
data.info()

<b> here I dealed with the missing values in the CustomerID column. First, I maked sure the there are no invoices with more than one customer. </b>

In [ ]:
invoices = data.groupby('InvoiceNo').agg({'CustomerID':'nunique'}).reset_index()
invoices.query('CustomerID > 1')

<b>I tried to fill the missing values in the CustomerID column in a similar method as I did with the description column. Now I used the invoice column. I followed the following logic: If there is an invoice to which a customer number is attached, I will add the customer number to all the rows that contain the invoice number. </b>

In [ ]:
non_missing_customers = data.query('CustomerID.notnull()')
top_custumer= non_missing_customers.groupby('InvoiceNo').agg({'CustomerID':'first'}).reset_index()
top_custumer.columns=['InvoiceNo', 'temp_cust']
data = data.merge(top_custumer, how='left', on='InvoiceNo')
data['CustomerID'] = data.CustomerID.fillna(data['temp_cust'])
data.info()

In [ ]:
print('Share of missing values in CustomerID column:', len(data.query('CustomerID.isnull()')) / len(data))

<b> The number of missing values in the CustomerID column remains the same because there is no invoice that has a customer number attached to it in some lines and not in others.
Since the missing values in this column make up a large percentage of the total data, and in most of the project the customer number is insignificant, for now we will leave the missing values as they are.</b>

In [ ]:
data = data.drop(labels=['temp_cust'], axis=1)

<b> Here I dealed with the negative values in Quantity and UnitPrice columns:</b><br>
<b> UnitPrice:</b>

In [ ]:
data.query('UnitPrice ==0')

<b> Products that sell for $ 0 are products that may be 'loss leaders'. Therfore, for now I will keep them as they are.</b>

In [ ]:
data.query('UnitPrice <0')

In [ ]:
data.query('StockCode =="B"')

<b> According to 'Investopedia' website, Bad debt is an expense that a business incurs once the repayment of credit previously extended to a customer is estimated to be uncollectible and is thus recorded as a charge off.</b><br>
[link to investopedia page](https://www.investopedia.com/terms/b/baddebt.asp)

<b>That is, this rows was entered by the business for a bureaucratic purpose. The purpose of this project is to learn about customer behavior so I decided to omit lines that contain this code.</b>

In [ ]:
data = data.query('StockCode !="B"')

<b> Quantity:</b>

In [ ]:
data.query('Quantity == 0')

<b> There are no rows with 0 in the quantity column, which is good. </b>

In [ ]:
print('Number of rows with a negative value in the quantity column:',len(data.query('Quantity < 0')))
data.query('Quantity < 0').head()


<b> The number of rows with a negative amount is pretty large.<br>
There are 2 reasonable options:<br> First option:  the minus was entered by mistake, i.e. the quantities are the absolute value of the existing value.<br>
Option Two: The negative quantities are actually credits of other items.

    
Since I can not determine with certainty for each line whether it is a discount or not, I decided to omit the blacks that contain a negative amount.</b>

In [ ]:
data = data.query('Quantity > 0')
data.shape

<b> While I checked the data I noticed that there are descriptions in Laowercase that seem to be for bureaucratic needs and not  purchases of real products</b>

In [ ]:
data.query('Description.str.islower()').head()

In [ ]:
data.query('Description.str.islower()').Description.unique()

In [ ]:
data.query('Description.str.islower()').UnitPrice.describe()

In [ ]:
# dropping rows with description in lowercase
data = data.query('~Description.str.islower()')
data.shape


### Dropping duplicates and correcting type of columns


In [ ]:
# converting column types
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
# dropping duplicates
data = data.drop_duplicates()

data.info()

In [ ]:
print('Size of data after preprocessing:', len(data))
print('The share of all deleted rows :', (len(data_1)-len(data)) / len(data_1))

## EDA:

In [ ]:
# adding column 'date' that contains only the date (without time)
data['date'] =  data['InvoiceDate'].dt.date
data.head()

In [ ]:
invoices_and_dates = data.groupby('InvoiceNo').agg({'date':['nunique', 'first']}).reset_index()
invoices_and_dates.columns = ['InvoiceNo', 'unique_dates', 'date']
invoices_and_dates.head()

In [ ]:
#making sure that all the rows of each invoice has the same date
invoices_and_dates.query('unique_dates > 1')

In [ ]:
print('The data contains invoices from {} to {}'.format(invoices_and_dates.date.min(),invoices_and_dates.date.max()))
print()
print('The average purchases (invoices) per day is:',float(invoices_and_dates.groupby('date').agg({'InvoiceNo':'nunique'}).mean()))

In [ ]:
fig = px.histogram(invoices_and_dates, x="date", title="Distribution of invoices during the year", 
                  )
fig.show()

<b> The data contains purchases between the dates 29.11.2018 - 1.12.2019. Between the  dates 23-29.12.2018 , no purchases were made , probably due to end-of-year holidays. The average purchase per day is 68 and it does not consider dates on which at least one purchase was not made.</b>

In [ ]:
data['purchase_amount'] = data['Quantity'] * data['UnitPrice']
data.head()

In [ ]:
customers = data.groupby('CustomerID').agg({'InvoiceNo':'nunique','purchase_amount':'sum'}).reset_index()
customers.columns = ['CustomerID', 'total_invoices', 'total_purchase_amount']
customers.head()

In [ ]:
fig = px.histogram(customers, x="total_invoices", title="Distribution of invoices per customer", 
                labels={ "total_invoices": "Invoices per customer"})
fig.update_xaxes(range=(0,50))
fig.show()

In [ ]:
customers.total_invoices.describe()

<b> The average number of invoices per customer is 2.</b>

In [ ]:
fig = px.histogram(customers, x="total_purchase_amount", title="Distribution of total purchase amount per customer",
                  labels={ "total_purchase_amount": "Total purchases amount $"})
fig.update_xaxes(range=(0,10000))
fig.show()

In [ ]:
customers.total_purchase_amount.describe()

<b> The average total purchase amount per customer is 668 dolars. It is important to note that there are customers whose total purchase amount is 0 because they skew the results.</b>

In [ ]:
invoices = data.groupby('InvoiceNo').agg({'StockCode':'nunique','Quantity':'sum', 'purchase_amount':'sum' }).reset_index()
invoices.columns = ['InvoiceNo', 'unique_prudocts', 'total_products', 'purchase_amount']
invoices.head()

In [ ]:
fig = px.histogram(invoices, x="purchase_amount", title="Distribution of total purchase amount per invoice",
                  labels={ "purchase_amount": "Total purchases amount $"})
fig.update_xaxes(range=(0,5000))
fig.show()

In [ ]:
invoices.purchase_amount.describe()

<b> The average purchase amount per invoice is 297$ </b>

In [ ]:
products = data.groupby('StockCode').agg({'Quantity':'sum','purchase_amount':'sum','Description':'first' }).reset_index()
products.columns = ['StockCode', 'total_quantity', 'purchase_amount', 'description']
products.head()

In [ ]:
fig = px.scatter(products, x="total_quantity",y="purchase_amount", title="Distribution of total purchase amount per product")
fig.update_xaxes(range=(0,20000))
fig.update_xaxes(range=(0,20000))
fig.show()

In [ ]:
products.describe()

<b> Each product sold an average of 379 units, for an average of 679$</b>

In [ ]:
top_10_products = products.sort_values(by='purchase_amount', ascending=False).head(10)
fig =go.Figure(go.Bar(x = top_10_products["description"], y=top_10_products["purchase_amount"]))
fig.update_layout(title='Top 10 proffitable products ',  yaxis = dict(title_text='Total purchase amount $'))
fig.show()

## Categorizing products:

### Splitting to categories

In order to perform a categorization I used a code written by [Jean Snyman](https://www.jeansnyman.com/posts/unsupervised-text-clustering-with-k-means/)

The categorization is done in the following steps:

1. Text was cleared of numbers and symbols.

2. Each description was converted to a numeric value according to the [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) measure.<br>

3. I set the number of clusters by using the [elbow method](https://en.wikipedia.org/wiki/Elbow_method_(clustering)):<br>
"In cluster analysis, the elbow method is a heuristic used in determining the number of clusters in a data set. The method consists of plotting the explained variation as a function of the number of clusters, and picking the elbow of the curve as the number of clusters to use."

4. Predictions were made.

In [ ]:
stemmer = PorterStemmer()

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
REMOVE_NUM = re.compile('[\d+]')
STOPWORDS = set(stopwords.words('english'))

In [ ]:
def clean_text(text):
    """
    text: a string
    return: modified initial string
    """
    # lowercase text
    text = text.lower() 

    # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    
    # Remove the XXXX values
    text = text.replace('x', '') 
    
    # Remove white space
    text = REMOVE_NUM.sub('', text)

    #  delete symbols which are in BAD_SYMBOLS_RE from text
    text = BAD_SYMBOLS_RE.sub('', text) 

    # delete stopwords from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) 
    
    # removes any words composed of less than 2 or more than 21 letters
    text = ' '.join(word for word in text.split() if (len(word) >= 2 and len(word) <= 21))

    # Stemming the words
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    
    return text

In [ ]:
# cleaning the descriptions
products['clean_desc'] = products['description'].apply(clean_text)
products.head()

In [ ]:
# converting the text to numeric value
vectorizer = TfidfVectorizer(sublinear_tf= True, min_df=10, norm='l2', ngram_range=(1, 2), stop_words='english')
X_train_vc = vectorizer.fit_transform(products['clean_desc'])

In [ ]:
# setting the clusters to 10, only for test
k_clusters = 10

score = []
for i in range(1,k_clusters + 1):
    kmeans = KMeans(n_clusters=i,init='k-means++',max_iter=300,n_init=5,random_state=0)
    kmeans.fit(X_train_vc)
    score.append(kmeans.inertia_)
plt.plot(range(1,k_clusters + 1 ),score)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('Score')
plt.savefig('elbow.png')
plt.show()

<b> Here you can see that the graph curves after 4 clusters. Therefore, I set the number of clusters to be 4</b>

In [ ]:
k_clusters = 4
#making predictions
model = KMeans(n_clusters=k_clusters, init='k-means++', n_init=10, max_iter=600, tol=0.000001, random_state=0)
model.fit(X_train_vc)

clusters = model.predict(X_train_vc)
# Create a new column to display the predicted result
products["cluster"] = clusters
products.head()

<b> Here I tried to figure out what What characterizes  each cluster </b>

In [ ]:
# printing the most common words of each cluster
order_centroids = model.cluster_centers_.argsort()[:, ::-1]

terms = vectorizer.get_feature_names()
for i in range(k_clusters):
    top_ten_words = [terms[ind] for ind in order_centroids[i, :k_clusters]]
    print("Cluster {}: {}".format(i, ' '.join(top_ten_words)))

In [ ]:
# adding cluster column to the main table
data = data.merge(products[['StockCode','cluster']], how='left', on='StockCode')
data.head()

### Study the clusters

In [ ]:
clusters = data.groupby('cluster').agg({'StockCode':'nunique','Quantity':'sum','purchase_amount':'sum' }).reset_index()
clusters.columns = ['cluster', 'unique_products','total_quantity','total_purchase_amount']
clusters['relative_quantity'] = clusters['total_quantity']  / clusters['unique_products']
clusters['relative_purchase_amount'] = clusters['total_purchase_amount']  / clusters['unique_products']
clusters

In [ ]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Share of unique products in each cluster", "Total purchase amount per cluster", "Relative quantity of sold pruducts", "Relative purchase amount"),
    specs=[[{"type": "pie"}, {"type": "pie"}],[{"type": "pie"}, {"type": "pie"}]])

fig.add_trace(go.Pie(values = clusters["unique_products"], labels=clusters["cluster"]),
              row=1, col=1)

fig.add_trace(go.Pie(values = clusters["total_purchase_amount"], labels=clusters["cluster"]),
              row=1, col=2)

fig.add_trace(go.Pie(values = clusters["relative_quantity"], labels=clusters["cluster"]),
              row=2, col=1)

fig.add_trace(go.Pie(values = clusters["relative_purchase_amount"], labels=clusters["cluster"]),
              row=2, col=2)

fig.show()

<b> Cluster 0 is the cluster that contains the most prudocts, and cluster 3 have the least products. Relative to the number of products it contains, cluster 2 is the most profitable of all the clusters, while cluster 3 is the least profitable.</b>


### Find the most and least profitable products of each category


In [ ]:
top_5_cluster_0 = products.query('cluster==0').sort_values(by='purchase_amount', ascending=False).head()
top_5_cluster_1 = products.query('cluster==1').sort_values(by='purchase_amount', ascending=False).head()
top_5_cluster_2 = products.query('cluster==2').sort_values(by='purchase_amount', ascending=False).head()
top_5_cluster_3 = products.query('cluster==3').sort_values(by='purchase_amount', ascending=False).head()

bottom_5_cluster_0 = products.query('cluster==0').sort_values(by='purchase_amount').head()
bottom_5_cluster_1 = products.query('cluster==1').sort_values(by='purchase_amount').head()
bottom_5_cluster_2 = products.query('cluster==2').sort_values(by='purchase_amount').head()
bottom_5_cluster_3 = products.query('cluster==3').sort_values(by='purchase_amount').head()


In [ ]:
fig = make_subplots(
    rows=1, cols=2)  

fig.add_trace(go.Bar(x = top_5_cluster_0["description"], y=top_5_cluster_0["purchase_amount"],name='The 5 most proffitable'),
              row=1, col=1)

fig.add_trace(go.Bar(x = bottom_5_cluster_0["description"], y=bottom_5_cluster_0["purchase_amount"], name='The 5 least proffitable'),
              row=1, col=2)

fig.update_layout(title='The most and least proffitable products of cluster 0',  yaxis = dict(title_text='Total purchase amount $'))


fig.show()

In [ ]:
fig = make_subplots(
    rows=1, cols=2)  

fig.add_trace(go.Bar(x = top_5_cluster_1["description"], y=top_5_cluster_1["purchase_amount"],name='The 5 most proffitable'),
              row=1, col=1)

fig.add_trace(go.Bar(x = bottom_5_cluster_1["description"], y=bottom_5_cluster_1["purchase_amount"], name='The 5 least proffitable'),
              row=1, col=2)

fig.update_layout(title='The most and least proffitable products of cluster 1',  yaxis = dict(title_text='Total purchase amount $'))


fig.show()

In [ ]:
fig = make_subplots(
    rows=1, cols=2)  

fig.add_trace(go.Bar(x = top_5_cluster_2["description"], y=top_5_cluster_2["purchase_amount"],name='The 5 most proffitable'),
              row=1, col=1)

fig.add_trace(go.Bar(x = bottom_5_cluster_2["description"], y=bottom_5_cluster_2["purchase_amount"], name='The 5 least proffitable'),
              row=1, col=2)

fig.update_layout(title='The most and least proffitable products of cluster 2',  yaxis = dict(title_text='Total purchase amount $'))


fig.show()

In [ ]:
fig = make_subplots(
    rows=1, cols=2)  

fig.add_trace(go.Bar(x = top_5_cluster_3["description"], y=top_5_cluster_3["purchase_amount"],name='The 5 most proffitable'),
              row=1, col=1)

fig.add_trace(go.Bar(x = bottom_5_cluster_3["description"], y=bottom_5_cluster_3["purchase_amount"], name='The 5 least proffitable'),
              row=1, col=2)

fig.update_layout(title='The most and least proffitable products of cluster 3',  yaxis = dict(title_text='Total purchase amount $'))


fig.show()


## Price analysis:


### Finding max and min price for each product

In [ ]:
products_new = data.groupby('StockCode').agg({'Description':'first','UnitPrice':['min','max']}).reset_index()
products_new.columns = ['StockCode', 'Description', 'min_price', 'max_price']
products_new['max_differ'] = products_new['max_price'] -products_new['min_price']
products_new.describe()


### Studing the products that has the biggest difference 


In [ ]:
products_new.sort_values(by='max_differ', ascending=False).head(15)

<b> Here you can see that there are aome products that are not 'real products'. These are products with a non-numeric code</b>


### Studing  the change of prices throughout the year


In [ ]:
# Dropping products with nun-numeric stock code
max_differ_products=products_new.sort_values(by='max_differ', ascending=False).head(15).query('StockCode.str.isnumeric()').StockCode

In [ ]:
df=data.query('StockCode in @max_differ_products')
fig = px.line(df, x="date", y="UnitPrice", color='Description', title="Changes of prices during the year: 10 products with the highest differences")
fig.show()

<b> The general trend shows that the price of products is falling towards the end of the year</b>

In [ ]:
products_new.query('max_price == 0')

In [ ]:
# dropping products whose maximum price is 0
data =data.merge(products_new.drop('Description', axis=1), how='left', on='StockCode')
data = data.query('max_price !=0')

# adding column 'perc_of_max_price' which calculate the percentage of the maximal price of the product
data['perc_of_max_price'] = data['UnitPrice'] /data['max_price'] * 100
data.head()

In [ ]:
fig = px.histogram(data, x="perc_of_max_price", title="Distribution of percentage of maximum price",
                  labels={ "perc_of_max_price": "Percentage of maximum price $"})

fig.show()

<b> Most purchases are purchased at a price that stands at 50% of the maximum price</b>

In [ ]:
data.perc_of_max_price.describe()


    
## Statistical Hypotheses Test

<b>H0: </b>There is no significant difference in the quantity of items purchased at a low price and items purchased at a high price<br>
<b>H1:</b>There is significant difference in the quantity of items purchased at a low price and items purchased at a high price

In [ ]:
low_price = data.query('perc_of_max_price<=50')
high_price = data.query('perc_of_max_price>50')

In [ ]:
print('The avergae quantity of products that are sold up to 50% of their maximal price:', low_price.Quantity.median())
print('The avergae quantity of products that are sold over 50% of their maximal price:', high_price.Quantity.median())

<b> Checking if the samples have a normal distribution</b>

In [ ]:
low_price.Quantity.describe()

In [ ]:
high_price.Quantity.describe()

<b> Since samples are not normally distributed I used the Whitney Man test</b>

In [ ]:
sample_1 = low_price.Quantity
sample_2 = high_price.Quantity

alpha = .05 #significance level

results = st.mannwhitneyu(sample_1, sample_2)

print('p-value: ', results.pvalue)

if (results.pvalue < alpha):
    print("Null hypothesis rejected: the difference is statistically significant")
else:
    print("Failed to reject the null hypothesis: we can't make conclusions about the difference") 


## Conclusion


<b>Changes of the data:</b>
* I filled in the missing values in the 'Description' column by using descriptions of products with the same stock code.</b>
* I dropped the rest of the rows with missing values in the 'Description' column.
* I dropped rows with lowercase description
* I drpped rows with negative value in the quantity column

<b> generak conclusions:</b>
* The average purchase per day is 68 
* The average number of invoices per customer is 2
* The average total purchase amount per customer is 668 dolars.
* The average purchase amount per invoice is 297
* Each product sold an average of 379 units, for an average of 679
* The general trend shows that the price of products is falling towards the end of the year
* Customers buy more in larger quantities when the unit price per product is lower

The main conclusion is not surprising: customers buy more in larger quantities when the unit price per product is lower.
Since I do not have data on the cost price of the beneficiaries, I do not have the ability to calculate cost versus profit so I refrain from giving recommendations.
